# Apresentação do trabalho

A idiea inicial do trabalho era montar um helper de TMP, baseado em uma dor particular. Ao investir de maneira amadora, é comum utilizarmos proporções aleatórias do portifólio entre ativos diversos. Eu particularmente só invisto em índices, mas não necessariamente faço a escolha ótima da distribuição entre eles para alcançar um bom risco-retorno. Dessa maneira a ideia inicial desse trabalho seria apresentar um site que a partir de dados inputados do usuário ele retornasse a proporção ótima dos ativos.

Essa tarefa foi concluida, entretanto não se encaixava nas formatações do trabalho. Dessa forma, a partir do MVP que eu tinha construido eu fiz alterações para adaptar ao trabalho. Assim, o usuário escolhe qual o grupo de ativos que ele quer analisar e então o código o entrega duas carteiras. 

Inicialmente, o projeto não necessitaria de um backtest, ja que o usuário iria usar as informações de previsão para o próximo período, entretando isso foi alterado, para que de fato o usuário veja o backtest

O trabalho usa as técnicas de clusterização para a partir dos ativos selecionados pelo usuário ele montar uma carteira de n ativos - decididos através do método do cotovelo. Então o usuário recebe três carteiras, uma ponderando as proporções pelo inverso da volatilidade e outra ponderando pelo retorno esperado. E a terceira implícita, que seria escolhendo proporções constantes, já que foram ativos decididos pela clusterização.

Há no código uma implementação pronta, mas comentada, de máximo sharpe, que não funcionou por problema de convexidade, reduzindo o número de carteiras de 4 para 3.

Perceba também que nada nesse notebook é feito pra funcionar exatamente, apenas para apresentar o que foi feito no trabalho em sí. Para rodar o trabalho, basta baixar os requirements e rodar o arquivo app.py, que funciona através do flask

# Leitura

A leitura de dados é feita a partir da função baixar_dados do arquivo otimizacao.py. Ela recebe alguns parâmetros, como a decisão de qual grupo de ativos. Que ficará mais claro no dicionário abaixo.

In [ ]:
def baixar_dados(tickers, start_date='2010-01-01', end_date='2025-01-01'):

    data = yf.download(tickers, start=start_date, end=end_date, progress=False).dropna(axis=1, how="all")


    prices = data['Close']

    prices = prices

    returns = prices.pct_change().dropna()

    #Pra que eu vou usar isso?
    retorno_acumulado = (1 + returns).cumprod()
    
    final_retorno_acumulado = retorno_acumulado.iloc[-1] if not retorno_acumulado.empty else pd.Series(1, index=data.columns)


    return data,returns,final_retorno_acumulado

## Limpeza

A base é dinâmicamente lida na aplicação, e limpa mais de uma vez, mas essencialmente na função baixar_dados. É importante entender que a função só faz sentida no contexto que é utlilizada.

## Escolha dos Ativos
Esse dicionário é o dicionário que contem os ativos possiveis. Ele é separado em 4 grupos de ativos: Ações Brasil, Ações EUA, ETFs Regionais e ETFs Macro. A escolha foi feita com base na decisão do usuário: se o usuário quiser manter X% do patrimônio em ações brasileiras, ele poderia utilizar X%*Proporções para chegar a sua carteira de ações brasileiras, de ações americanas, e assim por diante... Considerando-se grandes conjuntos de estratégias.

In [1]:
    mercados = {
        'BR': [
            # Financeiro
            "ITUB4.SA", "BBDC4.SA", "BBAS3.SA", "SANB11.SA", "BPAC11.SA",
            # Commodities / Energia
            "PETR4.SA", "VALE3.SA", "PRIO3.SA", "CSNA3.SA", "GGBR4.SA",
            "USIM5.SA", "SUZB3.SA", "KLBN11.SA", "BRKM5.SA",
            # Consumo
            "ABEV3.SA", "LREN3.SA", "MGLU3.SA", "VVAR3.SA", "ARZZ3.SA",
            "PCAR3.SA", "NTCO3.SA",
            # Infraestrutura e serviços
            "EQTL3.SA", "ELET3.SA", "ENGI11.SA", "CMIG4.SA", "TAEE11.SA",
            # Saúde
            "HAPV3.SA", "RADL3.SA", "FLRY3.SA",
            # Outros setores
            "RENT3.SA", "B3SA3.SA", "WEGE3.SA", "MULT3.SA", "CYRE3.SA",
        ],
        # 🇺🇸 Mercado Americano — Blue chips, techs e setores tradicionais
        'EUA': [
            # Tech e Comunicação
            "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA", "NFLX", "ADBE",
            # Financeiro
            "JPM", "GS", "BAC", "MS", "V", "MA", "BLK", "C",
            # Saúde
            "JNJ", "PFE", "UNH", "ABBV", "MRK", "TMO", "CVS",
            # Energia e Indústria
            "XOM", "CVX", "CAT", "BA", "GE", "MMM", "HON",
            # Consumo
            "PG", "KO", "PEP", "MCD", "DIS", "NKE", "COST", "WMT",
            # Diversificação
            "BRK-B", "INTC", "IBM", "ORCL", "AVGO"
        ],
        'ETFs Globais': [
                "SPY",     # S&P 500 (SPDR)
                "IVV",     # S&P 500 (iShares)
                "VOO",     # S&P 500 (Vanguard)
                "VTI",     # Total US Stock Market
                "VT",      # Global Total Market
                "VEA",     # Developed Markets (ex-US)
                "VWO",     # Emerging Markets
            ],

        'Conjunto ETFs Macro': [
                "XLK",     # Tecnologia
                "XLF",     # Financeiro
                "XLV",     # Saúde
                "XLE",     # Energia
                "XLY",     # Consumo Discricionário
                "XLP",     # Consumo Básico
                "XLI",     # Indústria
                "XLRE",    # Real Estate (Imobiliário)
                "XLC",     # Comunicação
                "SMH",     # Semicondutores
                "ARKK",    # Inovação disruptiva (Cathie Wood)
                "QQQ",     # Nasdaq-100
                "SKYY",    # Cloud computing
                "BOTZ",    # Robótica e IA
                "FINX",    # Fintechs
                "CLOU",    # Computação em nuvem
                "IEUR",    # Europa
                "EWG",     # Alemanha
                "EWU",     # Reino Unido
                "EWJ",     # Japão
                "EWW",     # México
                "EWH",     # Hong Kong
                "EWA",     # Austrália
                "EWC",     # Canadá
                "GLD",     # Ouro
                "SLV",     # Prata
                "USO",     # Petróleo WTI
                "DBA",     # Agricultura
                "UUP",     # Dólar americano
                "FXE",     # Euro
                "FXY",     # Iene
                "VNQ",     # Vanguard Real Estate
                "SCHH",    # Schwab US REIT
                "IYR",     # iShares US Real Estate
                "XLRE",    # Real Estate (SPDR)
            ]
        }

# Técnicas

A clueterizaçãa e montagem do portifólio são feitas por uma série de funções do arquivo otimizacao.py. (que aliás me orgulho da modularização)

A estrutura do código pode ser entendida através da função main_otimizacao (que obviamente não funciona nesse notebook)

In [2]:
def main_otimizacao(mercado,meses):

    tickers = mercados(mercado)

    inicio = separar_datas(meses)

    data, returns, final = baixar_dados(tickers,end_date=inicio)

    features, features_scaled = features_para_cluster(data, returns, final, meses)
    
    optimal_k, inertias, silhouette_scores= cluster_cotovelo(features_scaled)
    
    k_means, cluster = clusters(features_scaled, optimal_k)

    df_combinado = clusters_df(data,cluster,features,final)

    tickers_selecionados, retorno_acumulado_all, retorno_acumulado_selected = portfolio(returns, data, df_combinado, optimal_k)

    ret_med = returns[tickers_selecionados].mean()

    pesos_ret = ret_med / ret_med.sum()

    vols = returns[tickers_selecionados].std()

    pesos_inv_vol = (1/vols) / (1/vols).sum()

    gerar_graficos(features, cluster, returns, retorno_acumulado_all,
    retorno_acumulado_selected, df_combinado, silhouette_scores, list(range(2, 2 + len(inertias))), inertias,
    optimal_k, tickers_selecionados, pesos_inv_vol, pesos_ret, inicio, mercado)

    return pesos_inv_vol, pesos_ret, tickers_selecionados

# Artefato

O artefato foi feito em flask e tem basicamente 5 partes:
- index: o início
- formulario: personalização do usuário
- resultados: página de mostrando o portifólio
- detalhes: explicação e gráficos justificando a escolha

É importante citar que o trabalho, pela modularidade, é facilmente manipulável. Isso possibilita que melhorias futuras sejam bem fáceis.

# Fontes

Eu utilizei muita IA para reestruturar os códigos das aulas e modulariza-los. Para encontrar erros e preparar o trabalho. Então basicamente foi tudo feito por IA, baseado nos códigos das aulas e com muito esforço pra formatar no modelo aceitável no contexto de desenvolvimento.